In [1]:
import numpy as np
import pandas as pd
import os
import glob

In [17]:
# use glob to get all the csv files 
# in the folder
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "Files", "*.csv"))
num = 0

print(csv_files)

['/home/shuting/IDRs/Files/GO0003682_chromatin_binding_ PostIDR.csv', '/home/shuting/IDRs/Files/GO0003682_chromatin_binding_ Pred_df.csv', '/home/shuting/IDRs/Files/GO0003682_chromatin_binding_ Tstats_df.csv', '/home/shuting/IDRs/Files/GO0005730_nucleolus_ PostIDR.csv', '/home/shuting/IDRs/Files/GO0005730_nucleolus_ Pred_df.csv', '/home/shuting/IDRs/Files/GO0005730_nucleolus_ Tstats_df.csv', '/home/shuting/IDRs/Files/GO0030312_external_encapsulating_structure_ PostIDR.csv', '/home/shuting/IDRs/Files/GO0030312_external_encapsulating_structure_ Pred_df.csv', '/home/shuting/IDRs/Files/GO0030312_external_encapsulating_structure_ Tstats_df.csv', '/home/shuting/IDRs/Files/GO0031012_extracellular_matrix_ PostIDR.csv', '/home/shuting/IDRs/Files/GO0031012_extracellular_matrix_ Pred_df.csv', '/home/shuting/IDRs/Files/GO0031012_extracellular_matrix_ Tstats_df.csv', '/home/shuting/IDRs/Files/GO0042393_histone_binding_ PostIDR.csv', '/home/shuting/IDRs/Files/GO0042393_histone_binding_ Pred_df.csv',

In [ ]:
# loop over the list of csv files
for f in csv_files:
    
    # read the csv file
    df = pd.read_csv(f)
      
    # print the location and filename
    print('Location:', f)
    print('File Name:', f.split("\\")[-1])
    # print the content
    print('Content:')
    display(df)
    print()
    
    num +=1

In [16]:
num

15

In [18]:
name = "GO0003682_chromatin_binding_ PostIDR.csv"

In [24]:
name_split = name.split()
name_split

['GO0003682_chromatin_binding_', 'PostIDR.csv']

In [25]:
name_split[0]

'GO0003682_chromatin_binding_'

In [2]:
protein_df = pd.read_csv("../statistics_data/GO0000226_microtubule_cytoskeleton_organization_ Pred_df.csv",  names = ["Protein_name","Score"], header=0, index_col=0)
protein_df["Rank"] = protein_df["Score"].rank(ascending=False) 
protein_df = protein_df.sort_values("Rank")
protein_filter = protein_df.head(20)
protein_filter

,Score,Rank
Protein_name,,
P0CG33,0.496269,1.0
Q9NYA3,0.485431,2.0
A6NDK9,0.465245,3.0
Q5VT06,0.429136,4.0
Q9C0D2,0.373761,5.0
A6NDN3,0.331904,6.0
Q8N3K9,0.324916,7.0
H3BSY2,0.319907,8.0
A8MQT2,0.317983,9.0


In [3]:
filtered_protein_name = protein_filter.index.values
filtered_protein_name

array(['P0CG33', 'Q9NYA3', 'A6NDK9', 'Q5VT06', 'Q9C0D2', 'A6NDN3',
       'Q8N3K9', 'H3BSY2', 'A8MQT2', 'Q8IYY4', 'A7E2F4', 'Q8WYP5',
       'Q02952', 'Q8IYE1', 'H3BPF8', 'Q96SN8', 'P15311', 'Q9P219',
       'Q6NUN7', 'O95613'], dtype=object)

In [10]:
def compute_IDR_num(filter_IDRs):
    count_dict = dict()
    for ID in filter_IDRs["UniProtID"]:
        if ID in count_dict:
            count_dict[ID] += 1
        else:
            count_dict[ID] = 1

    count_list = []
    for ID in filter_IDRs["UniProtID"]:
        if ID in count_dict:
            count_list.append(count_dict.get(ID))
    filter_IDRs["The number of IDRs in each protein"] = count_list
    return count_list

In [20]:
IDR_df = pd.read_csv("../statistics_data/GO0000226_microtubule_cytoskeleton_organization_ PostIDR.csv", names = ["Protein IDR","Score"], header=0)

# Split GO_term string in order to let user understand the meaning of each column
# https://stackoverflow.com/questions/14745022/how-to-split-a-dataframe-string-column-into-two-columns
# n=the number of columns you expect
IDR_df[['UniProtID', 'Redundancy', 'The beginning position of IDR', 'The ending position of IDR']] = IDR_df['Protein IDR'].str.split('_', n=4, expand=True)

filter_IDRs = pd.DataFrame([])
for p_name in filtered_protein_name:
    for IDR_name in IDR_df["Protein IDR"]:
        if p_name in IDR_name:
            s2 = IDR_df[IDR_df["Protein IDR"] == IDR_name]
            filter_IDRs = pd.concat([filter_IDRs, s2], ignore_index=True)

# compute the total number of IDRs in each protein  
count_list = compute_IDR_num(filter_IDRs)
filter_IDRs["The number of IDRs in each protein"] = count_list

# # remove all IDRs whose score <= 0.5
# filter_IDRs = filter_IDRs[filter_IDRs["Score"] > 0.5]

# filter_IDRs = filter_IDRs.drop(columns=['Protein IDR', 'Redundancy'])
# filter_IDRs = filter_IDRs[['UniProtID', 'The beginning position of IDR', 'The ending position of IDR', 'Score', 'The number of IDRs in each protein']]

# # if the protein has more than 2 IDRs, remove IDRs whose score <= 0.6
# temp_df = filter_IDRs[filter_IDRs["The number of IDRs in each protein"]>2]
# filter_IDRs = filter_IDRs.drop((temp_df["Score"] < 0.6).index.values)

# # hold two digits after the decimal points
# filter_IDRs["Score"] = filter_IDRs["Score"].astype(float)
# filter_IDRs["Score"] = filter_IDRs["Score"].round(2)

filter_IDRs

,Protein IDR,Score,UniProtID,Redundancy,The beginning position of IDR,The ending position of IDR,The number of IDRs in each protein
0,P0CG33_IDR_1_693,1.000000,P0CG33,IDR,1,693,1
1,Q9NYA3_IDR_1_693,1.000000,Q9NYA3,IDR,1,693,1
2,A6NDK9_IDR_1_693,1.000000,A6NDK9,IDR,1,693,1
3,Q5VT06_IDR_1_2842,0.890458,Q5VT06,IDR,1,2842,2
4,Q5VT06_IDR_2876_2908,0.109542,Q5VT06,IDR,2876,2908,2
5,Q9C0D2_IDR_1_2537,1.000000,Q9C0D2,IDR,1,2537,1
6,A6NDN3_IDR_1_197,0.428248,A6NDN3,IDR,1,197,2
7,A6NDN3_IDR_213_693,0.571752,A6NDN3,IDR,213,693,2
8,Q8N3K9_IDR_1_3505,1.000000,Q8N3K9,IDR,1,3505,1
9,H3BSY2_IDR_176_601,0.829767,H3BSY2,IDR,176,601,2


In [72]:
count_list = []
for ID in filter_IDRs["UniProtID"]:
    if ID in counts:
        count_list.append(counts.get(ID))
len(count_list)

36

In [24]:
df = pd.read_csv("../statistics_data/GO1990904_ribonucleoprotein_complex_ Pred_df.csv")
df

,Unnamed: 0,GO1990904_ribonucleoprotein_complex
0,Q5VTL8,0.031361
1,Q5VTQ0,0.011377
2,Q5VTR2,0.028818
3,Q5VUJ6,0.027855
4,Q5VUM1,0.045922
...,...,...
11635,Q8NEH6,0.030196
11636,Q8NEJ9,0.085473
11637,Q8NEK5,0.016681
11638,Q8NEL0,0.017471


In [25]:
df["Rank"] = df["GO1990904_ribonucleoprotein_complex"].rank(ascending=False) 
df = df.sort_values("Rank")
df = df.head(20)
df

,Unnamed: 0,GO1990904_ribonucleoprotein_complex,Rank
10296,Q14152,0.415230,1.0
3843,P08621,0.331117,2.0
8143,Q9Y3X0,0.319006,3.0
4263,P23588,0.302760,4.0
8018,Q9UPT8,0.258572,5.0
11178,Q86VM9,0.214887,6.0
10403,Q15424,0.210464,7.0
11593,Q8NC51,0.208521,8.0
7062,Q9H1E3,0.203369,9.0
7898,Q9UKV3,0.202179,10.0


In [26]:
df.index.values

array([10296,  3843,  8143,  4263,  8018, 11178, 10403, 11593,  7062,
        7898,  8473, 10475,  2194,  8099,  9308, 11092,   763, 10246,
        6624, 10647])